In [1]:
import numpy as np
import pandas as pd
import pickle as pkl
import xarray as xr
import copy
import os
import sys
sys.path.append(os.path.realpath('./split-data/'))
import tools
import metrics
import random

import matplotlib 
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable


In [2]:
xr.__version__

'0.16.1'

In [3]:

    
f1='/home/gsnearing/projects/mclstm_paper/experiment/runs/lstm/standard_training/'
f2='lstm_standard_training_0_1605_101922/test/model_epoch030/'
f3='ensemble_results.p'
with open(f1+f3, 'rb') as fb:
    lstm_results_ens = pkl.load(fb)
    
f1='/home/gsnearing/projects/mclstm_paper/experiment/runs/'
f2='lstm/standard_training/lstm_standard_training_0_1605_101922/test/model_epoch030/'
f3='test_results.p'
with open(f1+f2+f3, 'rb') as fb:
    lstm_results_0 = pkl.load(fb)
    
f1='/home/gsnearing/projects/mclstm_paper/experiment/runs/lstm/hp_training/'
f2='lstm_hp_training_0_1505_010409/test/model_epoch030/'
f3='ensemble_results.p'
with open(f1+f3, 'rb') as fb:
    hplstm_results_ens = pkl.load(fb)
    

# f1='/home/gsnearing/projects/mclstm_paper/experiment/runs/mclstm/standard_training/ensemble_results.p'
# with open(f1, 'rb') as fb:
#     mclstm_results_ens = pkl.load(fb)
    
f1='/home/gsnearing/projects/mclstm_paper/experiment/runs/'
f2='mclstm/standard_training/mclstm_standard_training_0_2205_120945/test/model_epoch030/test_results.p'
with open(f1+f2, 'rb') as fb:
    mclstm_results_0 = pkl.load(fb)

# SACSMA Runs from Grey
with open('/home/jmframe/data/sacsma_camels_daymet/ensemble_means_sac_for_mc_paper_standard.pkl', 'rb') as fb:
    sacsma_results_ens = pkl.load(fb)
# SACSMA Runs from Grey
with open('/home/jmframe/data/sacsma_camels_daymet/single_value_sac_for_mc_paper_standard.pkl', 'rb') as fb:
    sacsma_results_0 = pkl.load(fb)
# SACSMA Runs from Andy Newman
with open('/home/jmframe/data/sacsma_camels_daymet/ensemble_means.pkl', 'rb') as fb:
    sacsma_results_ucar = pkl.load(fb)

In [4]:
# Convert flow to   CFS mm -> ft     km^2 -> ft^2    hr->s
conversion_factor = 0.00328084 * 10763910.41671 / 3600 / 24

In [5]:
# Get a list of all the basins in the analysis
basin_list = lstm_results_ens.keys()

# Camels attributes with RI information
dataName = 'ri-calc.csv'
# load the data with pandas
pd_attributes = pd.read_csv(dataName, sep=',', index_col='gauge_id')

# Add the basin ID as a 8 element string with a leading zero if neccessary
basin_id_str = []
for a in pd_attributes.index.values:
    basin_id_str.append(str(a).zfill(8))
pd_attributes['basin_id_str'] = basin_id_str

open_this_dir = "/home/NearingLab/projects/jmframe/lstm_camels_NWM_post_processing_paper_april2021/"
openthis = open_this_dir+'data/calibrated_basins.txt'
calibrated_basins = pd.read_csv(openthis)
cb = calibrated_basins['Domain_Meta.gage_id'].values
pd_attributes['calibrated'] = [np.nan]*pd_attributes.shape[0]
calibrated_basins_list = []
for ib, b in enumerate(basin_list):
    if b in cb:
        calibrated_basins_list.append(b)
        pd_attributes.loc[int(b), 'calibrated'] = True
    else:
        pd_attributes.loc[int(b), 'calibrated'] = False

In [6]:
def calculate_all_metrics_for_frequency_analysis(analysis_dict, flows):
    
    sims = list(flows.keys())[:-1]

    for metric in metrics.get_available_metrics():

        score = {sim:0 for sim in sims}
        
        if metric == 'NSE':
            for sim in sims:
                score[sim] = metrics.nse(flows['obs'],flows[sim])
        if metric == 'MSE':
            for sim in sims:
                score[sim] = metrics.mse(flows['obs'],flows[sim])
        if metric == 'RMSE':
            for sim in sims:
                 score[sim] = metrics.rmse(flows['obs'],flows[sim])
        if metric == 'KGE':
            for sim in sims:
                score[sim] = metrics.kge(flows['obs'],flows[sim])
        if metric == 'Alpha-NSE':
            for sim in sims:
                score[sim] = metrics.alpha_nse(flows['obs'],flows[sim])
        if metric == 'Beta-NSE':
            for sim in sims:
                score[sim] = metrics.beta_nse(flows['obs'],flows[sim])
        if metric == 'Pearson-r':
            for sim in sims:
                score[sim] = metrics.pearsonr(flows['obs'],flows[sim])
        if metric == 'Peak-Timing':
            for sim in sims:
                score[sim] = np.abs(metrics.mean_peak_timing(flows['obs'],flows[sim]))
        if metric == 'FHV':
            for sim in sims:
                score[sim] = metrics.fdc_fhv(flows['obs'],flows[sim])          
        if metric == 'FLV':
            for sim in sims:
                score[sim] = metrics.fdc_flv(flows['obs'],flows[sim])          
        if metric == 'FMS':
            for sim in sims:
                score[sim] = metrics.fdc_fms(flows['obs'],flows[sim])          
          
        for sim in sims:
            analysis_dict[metric][sim].append(score[sim])
        
    return

In [7]:

#-------------------------------------------------------------------------------------------------
# Set up lists to use in loops
models =        ['lstm','mc', 'sac']
flows =         ['lstm','mc', 'sac', 'obs']
#-------------------------------------------------------------------------------------------------

#------------------------------------------------------------------------------------------------- 
breaktheloop=False
#-------------------------------------------------------------------------------------------------         

#-------------------------------------------------------------------------------------------------
load_data=True
dump_data=False
file_name = 'frequency_analysis_dict_standard.pkl'
if load_data:
    with open(file_name, 'rb') as fb:
        analysis_dict_all = pkl.load(fb)
else:
    analysis_dict_all = {}
#-------------------------------------------------------------------------------------------------
            

    #-------------------------------------------------------------------------------------------------
    #-----LOOP THROUGH BASINS------------------------------------------------------------------------
    #-------------------------------------------------------------------------------------------------

    for ib, basin_0str in enumerate(basin_list): 
        basin_int = int(basin_0str)


        #-------------------------------------------------------------------------------------------------
        # Setting up the dictionary for the single basin results. Then will add to the overall dict.
        analysis_dict = {metric:{model:[] for model in models} for metric in metrics.get_available_metrics()}
        #-------------------------------------------------------------------------------------------------


        #-------------------------------------------------------------------------------------------------
        # We need the basin area to convert to CFS, to interpolate the RI from LPIII
        basin_area = pd_attributes.loc[basin_int, 'area_geospa_fabric']
        basin_str = tools.gauge_id_str(basin_int)
        #-------------------------------------------------------------------------------------------------


        #-------------------------------------------------------------------------------------------------
        #----  Set the time period for metrics   ---------------------------------------------------------
        date_from = '1989-10'
        date_to = '1999-09'
        #-------------------------------------------------------------------------------------------------    

        #-------------------------------------------------------------------------------------------------
        # Make dictionary with all the flows
        flow_mm = {}
        #-------------------------------------------------------------------------------------------------


        #-------------------------------------------------------------------------------------------------
        # Standard LSTM data trained on all years
        xrr = lstm_results_0[basin_0str]['1D']['xr']['QObs(mm/d)_sim']
        flow_mm['lstm'] = pd.DataFrame(data=xrr.values,index=xrr.date.values).loc[date_from:date_to]
        #-------------------------------------------------------------------------------------------------        
        # Mass Conserving LSTM data
        xrr = mclstm_results_0[basin_0str]['1D']['xr']['QObs(mm/d)_sim']
        flow_mm['mc'] = pd.DataFrame(data=xrr.values,index=xrr.date.values).loc[date_from:date_to]
        #-------------------------------------------------------------------------------------------------
        # SACSMA Sinlge run
        df = sacsma_results_0[basin_0str]
        flow_mm['sac'] = df.loc[date_from:date_to]
        #-------------------------------------------------------------------------------------------------
        # OBSERVATIONS
        xrr = mclstm_results_0[basin_0str]['1D']['xr']['QObs(mm/d)_obs']
        flow_mm['obs'] = pd.DataFrame(data=xrr.values,index=xrr.date.values).loc[date_from:date_to]
        #-------------------------------------------------------------------------------------------------


        #-------------------------------------------------------------------------------------------------
        # Make all xarray data similar
        for iflow in flows:
            if iflow == 'sac': #already in the correct format
                flow_mm[iflow] = xr.DataArray(np.array(flow_mm[iflow].values, dtype='float32'), 
                               coords=dict(datetime=flow_mm[iflow].index.values), dims=['datetime'])
            else:
                flow_mm[iflow] = xr.DataArray(flow_mm[iflow].values[:,0], 
                               coords=dict(datetime=flow_mm[iflow].index.values), dims=['datetime'])
        #-------------------------------------------------------------------------------------------------


        #-------------------------------------------------------------------------------------------------
        calculate_all_metrics_for_frequency_analysis(analysis_dict, flow_mm)
        #-------------------------------------------------------------------------------------------------


        #-------------------------------------------------------------------------------------------------     
        if breaktheloop:
            break
        #-------------------------------------------------------------------------------------------------     

        #-------------------------------------------------------------------------------------------------
        #Now that the basin has been analyzed successfully, add it to the larger dictionary
        analysis_dict_all[basin_0str] = analysis_dict
        #------------------------------------------------------------------------------------------------- 


    #-------------------------------------------------------------------------------------------------
    if dump_data:
        with open(file_name, 'wb') as fb:
            pkl.dump(analysis_dict_all, fb)


In [8]:
analysis_dict = {metric:{model:[] for model in models} for metric in metrics.get_available_metrics()}
for ib, basin_0str in enumerate(basin_list):
    try:
        for metric in metrics.get_available_metrics():
            for model in models:
                analysis_dict[metric][model].extend(analysis_dict_all[basin_0str][metric][model])
    except:
        continue

In [9]:
for metric in metrics.get_available_metrics():
    print(metric)
    for model in models:
        print(model, np.nanmean(analysis_dict[metric][model]))
        print(model, np.nanmedian(analysis_dict[metric][model]))
        print(model, np.nanmax(analysis_dict[metric][model]))
        print(model, np.nanmin(analysis_dict[metric][model]))

NSE
lstm 0.701801991226983
lstm 0.7466343641281128
lstm 0.9495692849159241
lstm -0.7543091773986816
mc 0.7002821243780256
mc 0.7403706610202789
mc 0.933014914393425
mc 0.03943204879760742
sac 0.4326427679810596
sac 0.6132773756980896
sac 0.9040184915065765
sac -46.69303512573242
MSE
lstm 3.0440030224014447
lstm 1.5631946325302124
lstm 44.64142990112305
lstm 0.0036376677453517914
mc 3.0576072131812384
mc 1.6076539754867554
mc 40.63725280761719
mc 0.002753547625616193
sac 4.5934562918185495
sac 2.2922158241271973
sac 90.3749771118164
sac 0.002935725497081876
RMSE
lstm 1.4583408709397123
lstm 1.2502778221380288
lstm 6.681424241965409
lstm 0.06031308104674964
mc 1.4660073019791944
mc 1.2679329538610293
mc 6.3747355088362045
mc 0.05247425678955532
sac 1.7872247164583677
sac 1.5140065469234925
sac 9.506575467107828
sac 0.05418233565546871
KGE
lstm 0.7095393665062932
lstm 0.7548714407165988
lstm 0.9541482623731873
lstm -2.418916341400373
mc 0.7019183324721519
mc 0.750711524974042
mc 0.9552172

In [10]:
table_metrics = ['NSE','KGE','Pearson-r','Alpha-NSE','Beta-NSE','FHV','FLV','FMS']
print('NSE (mean), {}, {}, {}'.format(np.round(np.nanmean(analysis_dict['NSE']['lstm']),4),
                                          np.round(np.nanmean(analysis_dict['NSE']['mc']),4),
                                          np.round(np.nanmean(analysis_dict['NSE']['sac']),4)))
for metric in table_metrics:
    print('{} (median), {}, {}, {}'.format(metric,
                                          np.round(np.nanmedian(analysis_dict[metric]['lstm']),4),
                                          np.round(np.nanmedian(analysis_dict[metric]['mc']),4),
                                          np.round(np.nanmedian(analysis_dict[metric]['sac']),4)))
print('Peak-Timing (median), {}, {}, {}'.format(np.round(np.nanmedian(analysis_dict['NSE']['lstm']),4),
                                          np.round(np.nanmedian(analysis_dict['NSE']['mc']),4),
                                          np.round(np.nanmedian(analysis_dict['NSE']['sac']),4)))

NSE (mean), 0.7018, 0.7003, 0.4326
NSE (median), 0.7466, 0.7404, 0.6133
KGE (median), 0.7549, 0.7507, 0.6018
Pearson-r (median), 0.8772, 0.8735, 0.8162
Alpha-NSE (median), 0.8645, 0.842, 0.7862
Beta-NSE (median), -0.0367, -0.0291, 0.0669
FHV (median), -14.3602, -15.494, -19.1006
FLV (median), 12.9475, -29.8015, 48.8138
FMS (median), -3.7037, -7.8063, -38.9932
Peak-Timing (median), 0.7466, 0.7404, 0.6133


# ---------------------------------
# NWM Time Split

# NWM Time Split
# ---------------------------------

In [11]:
# NWM Reanalysis
f='/home/NearingLab/projects/jmframe/lstm_camels_NWM_post_processing_paper_april2021/data/nwm_chrt_v2_1d_local.p'
with open(f, 'rb') as fb:
    nwm_results = pkl.load(fb)

f1='/home/gsnearing/projects/mclstm_paper/experiment/runs/lstm/nwm_training/ensemble_results.p'
with open(f1, 'rb') as fb:
    lstm_results_ens = pkl.load(fb)
    
f1='/home/gsnearing/projects/mclstm_paper/experiment/runs/'
f2='lstm/nwm_training/lstm_nwm_training_9_2405_151025/test/model_epoch030/'
f3='test_results.p'
with open(f1+f2+f3, 'rb') as fb:
    lstm_results_9 = pkl.load(fb)
    
# f1='/home/gsnearing/projects/mclstm_paper/experiment/runs/mclstm/standard_training/ensemble_results.p'
# with open(f1, 'rb') as fb:
#     mclstm_results_ens = pkl.load(fb)
    
f1='/home/gsnearing/projects/mclstm_paper/experiment/runs/'
f2='mclstm/nwm_training/mclstm_nwm_training_0_2205_121002/test/model_epoch030/test_results.p'
with open(f1+f2, 'rb') as fb:
    mclstm_results_0 = pkl.load(fb)


# SACSMA Runs from Andy Newman
with open('/home/jmframe/data/sacsma_camels_daymet/ensemble_means.pkl', 'rb') as fb:
    sacsma_results_ucar = pkl.load(fb)
    
# SACSMA Runs from Grey
with open('/home/jmframe/data/sacsma_camels_daymet/ensemble_means_sac_for_mc_paper_nwm.pkl', 'rb') as fb:
    sacsma_results_ens = pkl.load(fb)
# SACSMA Runs from Grey
with open('/home/jmframe/data/sacsma_camels_daymet/single_value_sac_for_mc_paper_nwm.pkl', 'rb') as fb:
    sacsma_results_0 = pkl.load(fb)

In [12]:
#-------------------------------------------------------------------------------------------------
# Set up lists to use in loops
models =        ['lstm','mc', 'sac', 'nwm']
flows =         ['lstm','mc', 'sac', 'nwm', 'obs']
#-------------------------------------------------------------------------------------------------

#------------------------------------------------------------------------------------------------- 
breaktheloop=False
#-------------------------------------------------------------------------------------------------         

#-------------------------------------------------------------------------------------------------
load_data=True
dump_data=False
file_name = 'frequency_analysis_dict_nwm.pkl'
if load_data:
    with open(file_name, 'rb') as fb:
        analysis_dict_all = pkl.load(fb)
else:
    analysis_dict_all = {}
#-------------------------------------------------------------------------------------------------            

    #-------------------------------------------------------------------------------------------------
    #-----LOOP THROUGH BASINS------------------------------------------------------------------------
    #-------------------------------------------------------------------------------------------------

    for ib, basin_0str in enumerate(basin_list): 
        basin_int = int(basin_0str)


        #-------------------------------------------------------------------------------------------------
        # Get the NWM data for this basin in an xarray dataset.
        xr_nwm = xr.DataArray(nwm_results[basin_0str]['streamflow'].values, 
                 coords=[nwm_results[basin_0str]['streamflow'].index], 
                 dims=['datetime'])
        #-------------------------------------------------------------------------------------------------


        #-------------------------------------------------------------------------------------------------
        # Setting up the dictionary for the single basin results. Then will add to the overall dict.
        analysis_dict = {metric:{model:[] for model in models} for metric in metrics.get_available_metrics()}
        #-------------------------------------------------------------------------------------------------


        #-------------------------------------------------------------------------------------------------
        # We need the basin area to convert to CFS, to interpolate the RI from LPIII
        basin_area = pd_attributes.loc[basin_int, 'area_geospa_fabric']
        basin_str = tools.gauge_id_str(basin_int)
        #-------------------------------------------------------------------------------------------------


        #-------------------------------------------------------------------------------------------------
        #----  Set the time period for metrics   ---------------------------------------------------------
        date_from = '1996-10'
        date_to = '2014-09'
        #-------------------------------------------------------------------------------------------------    

        #-------------------------------------------------------------------------------------------------
        # Make dictionary with all the flows
        flow_mm = {}
        #-------------------------------------------------------------------------------------------------


        #-------------------------------------------------------------------------------------------------        
         # NWM data
        sim_nwm = xr_nwm.loc[date_from:date_to]
        # convert from CFS to mm/day
        # fm3/s * 3600 sec/hour * 24 hour/day / (m2 * mm/m)
        flow_mm['nwm'] = sim_nwm*3600*24/(basin_area*1000)
        #-------------------------------------------------------------------------------------------------
        # Standard LSTM data trained on all years
        xrr = lstm_results_9[basin_0str]['1D']['xr']['QObs(mm/d)_sim']
        flow_mm['lstm'] = pd.DataFrame(data=xrr.values,index=xrr.date.values).loc[date_from:date_to]
        #-------------------------------------------------------------------------------------------------        
        # Mass Conserving LSTM data
        xrr = mclstm_results_0[basin_0str]['1D']['xr']['QObs(mm/d)_sim']
        flow_mm['mc'] = pd.DataFrame(data=xrr.values,index=xrr.date.values).loc[date_from:date_to]
        #-------------------------------------------------------------------------------------------------
        # SACSMA Sinlge run
        df = sacsma_results_0[basin_0str]
        flow_mm['sac'] = df.loc[date_from:date_to]
        #-------------------------------------------------------------------------------------------------
        # OBSERVATIONS
        xrr = mclstm_results_0[basin_0str]['1D']['xr']['QObs(mm/d)_obs']
        flow_mm['obs'] = pd.DataFrame(data=xrr.values,index=xrr.date.values).loc[date_from:date_to]
        #-------------------------------------------------------------------------------------------------


        #-------------------------------------------------------------------------------------------------
        # Make all xarray data similar
        for iflow in flows:
            if iflow == 'nwm': #already in the correct format
                continue
            if iflow == 'sac': #already in the correct format
                flow_mm[iflow] = xr.DataArray(np.array(flow_mm[iflow].values, dtype='float32'), 
                               coords=dict(datetime=flow_mm[iflow].index.values), dims=['datetime'])
            else:
                flow_mm[iflow] = xr.DataArray(flow_mm[iflow].values[:,0], 
                               coords=dict(datetime=flow_mm[iflow].index.values), dims=['datetime'])
        #-------------------------------------------------------------------------------------------------


        #-------------------------------------------------------------------------------------------------
        calculate_all_metrics_for_frequency_analysis(analysis_dict, flow_mm)
        #-------------------------------------------------------------------------------------------------


        #-------------------------------------------------------------------------------------------------     
        if breaktheloop:
            break
        #-------------------------------------------------------------------------------------------------     

        #-------------------------------------------------------------------------------------------------
        #Now that the basin has been analyzed successfully, add it to the larger dictionary
        analysis_dict_all[basin_0str] = analysis_dict
        #------------------------------------------------------------------------------------------------- 


    #-------------------------------------------------------------------------------------------------
    if dump_data:
        with open(file_name, 'wb') as fb:
            pkl.dump(analysis_dict_all, fb)

In [13]:
analysis_dict = {metric:{model:[] for model in models} for metric in metrics.get_available_metrics()}
for ib, basin_0str in enumerate(basin_list):
    try:
        for metric in metrics.get_available_metrics():
            for model in models:
                analysis_dict[metric][model].extend(analysis_dict_all[basin_0str][metric][model])
    except:
        continue

In [14]:
for metric in metrics.get_available_metrics():
    print(metric)
    for model in models:
        print(model, np.nanmean(analysis_dict[metric][model]))
        print(model, np.nanmedian(analysis_dict[metric][model]))
        print(model, np.nanmax(analysis_dict[metric][model]))
        print(model, np.nanmin(analysis_dict[metric][model]))

NSE
lstm 0.677625152442022
lstm 0.7377163767814636
lstm 0.9539243802428246
lstm -3.5036845207214355
mc 0.6871839148008217
mc 0.7357942759990692
mc 0.9347539097070694
mc -0.7074671983718872
sac 0.4740075276789243
sac 0.5939785242080688
sac 0.8951981216669083
sac -17.777503967285156
nwm 0.4832593949281591
nwm 0.6265624834784564
nwm 0.9064579793112013
nwm -7.529840021384679
MSE
lstm 2.8266084784005634
lstm 1.3882181644439697
lstm 36.8264045715332
lstm 0.0012992407428100705
mc 2.8334937287501556
mc 1.3543530702590942
mc 38.59701919555664
mc 0.001101343659684062
sac 3.983947433828571
sac 2.2186665534973145
sac 53.10551834106445
sac 0.0011999034322798252
nwm 3.702273035179129
nwm 2.0008156295362562
nwm 51.702564737732224
nwm 0.0015889402395472514
RMSE
lstm 1.3839712402304314
lstm 1.1782267033317355
lstm 6.068476297352837
lstm 0.0360449822140346
mc 1.3837206894369705
mc 1.1637667593891374
mc 6.212649933446809
mc 0.0331864981533765
sac 1.67979530381413
sac 1.4895189000134623
sac 7.287353315234

In [15]:
table_metrics = ['NSE','KGE','Pearson-r','Alpha-NSE','Beta-NSE','FHV','FLV','FMS']
print('NSE (mean), {}, {}, {}, {}'.format(np.round(np.nanmean(analysis_dict['NSE']['lstm']),4),
                                          np.round(np.nanmean(analysis_dict['NSE']['mc']),4),
                                          np.round(np.nanmean(analysis_dict['NSE']['sac']),4),
                                          np.round(np.nanmean(analysis_dict['NSE']['nwm']),4)))
for metric in table_metrics:
    print('{} (median), {}, {}, {}, {}'.format(metric,
                                          np.round(np.nanmedian(analysis_dict[metric]['lstm']),4),
                                          np.round(np.nanmedian(analysis_dict[metric]['mc']),4),
                                          np.round(np.nanmedian(analysis_dict[metric]['sac']),4),
                                          np.round(np.nanmedian(analysis_dict[metric]['nwm']),4)))
print('Peak-Timing (mean), {}, {}, {}, {}'.format(np.round(np.nanmean(analysis_dict['NSE']['lstm']),4),
                                          np.round(np.nanmean(analysis_dict['NSE']['mc']),4),
                                          np.round(np.nanmean(analysis_dict['NSE']['sac']),4),
                                          np.round(np.nanmean(analysis_dict['NSE']['nwm']),4)))

NSE (mean), 0.6776, 0.6872, 0.474, 0.4833
NSE (median), 0.7377, 0.7358, 0.594, 0.6266
KGE (median), 0.7792, 0.7658, 0.5644, 0.6729
Pearson-r (median), 0.8767, 0.8761, 0.8101, 0.8177
Alpha-NSE (median), 0.9619, 0.9132, 0.8845, 0.8494
Beta-NSE (median), 0.0342, 0.0265, 0.1281, -0.0143
FHV (median), -2.5133, -7.1686, -8.568, -13.0238
FLV (median), -14.7745, -23.1853, 40.3737, 2.8542
FMS (median), -9.2581, -11.942, -29.8244, -5.2261
Peak-Timing (mean), 0.6776, 0.6872, 0.474, 0.4833


# ---------------------------------
# Return period split


# Return period split
# ---------------------------------

In [7]:
# NWM Reanalysis
f='/home/NearingLab/projects/jmframe/lstm_camels_NWM_post_processing_paper_april2021/data/nwm_chrt_v2_1d_local.p'
with open(f, 'rb') as fb:
    nwm_results = pkl.load(fb)

f1='/home/gsnearing/projects/mclstm_paper/experiment/runs/lstm/hp_training/ensemble_results.p'
with open(f1, 'rb') as fb:
    lstm_results_ens = pkl.load(fb)
    
f1='/home/gsnearing/projects/mclstm_paper/experiment/runs/'
f2='lstm/hp_training/lstm_hp_training_0_1505_010409/test/model_epoch030/'
f3='test_results.p'
with open(f1+f2+f3, 'rb') as fb:
    lstm_results_0 = pkl.load(fb)
    
# f1='/home/gsnearing/projects/mclstm_paper/experiment/runs/mclstm/standard_training/ensemble_results.p'
# with open(f1, 'rb') as fb:
#     mclstm_results_ens = pkl.load(fb)
    
f1='/home/gsnearing/projects/mclstm_paper/experiment/runs/'
f2='mclstm/hp_training/mclstm_hp_training_0_2205_120953/test/model_epoch030/test_results.p'
with open(f1+f2, 'rb') as fb:
    mclstm_results_0 = pkl.load(fb)


# SACSMA Runs from Andy Newman
with open('/home/jmframe/data/sacsma_camels_daymet/ensemble_means.pkl', 'rb') as fb:
    sacsma_results_ucar = pkl.load(fb)
    
# SACSMA Runs from Grey
with open('/home/jmframe/data/sacsma_camels_daymet/ensemble_means_sac_for_mc_paper_hp.pkl', 'rb') as fb:
    sacsma_results_ens = pkl.load(fb)
# SACSMA Runs from Grey
with open('/home/jmframe/data/sacsma_camels_daymet/single_value_sac_for_mc_paper_hp.pkl', 'rb') as fb:
    sacsma_results_0 = pkl.load(fb)

In [8]:
with open('./split-data/per_basin_train_periods_file.plk', 'rb') as fb:
    hp_train_periods = pkl.load(fb)
with open('./split-data/per_basin_val_periods_file.plk', 'rb') as fb:
    hp_val_periods = pkl.load(fb)
with open('./split-data/per_basin_test_periods_file.plk', 'rb') as fb:
    hp_test_periods = pkl.load(fb)

In [10]:
#-------------------------------------------------------------------------------------------------
# Set up lists to use in loops
models =        ['lstm','mc', 'sac', 'nwm']
flows =         ['lstm','mc', 'sac', 'nwm', 'obs']
#-------------------------------------------------------------------------------------------------

#------------------------------------------------------------------------------------------------- 
breaktheloop=False
#-------------------------------------------------------------------------------------------------         

#-------------------------------------------------------------------------------------------------
load_data=False
dump_data=True
file_name = 'frequency_analysis_dict_hp.pkl'
if load_data:
    with open(file_name, 'rb') as fb:
        analysis_dict_all = pkl.load(fb)
else:
    analysis_dict_all = {}
#-------------------------------------------------------------------------------------------------
            

    #-------------------------------------------------------------------------------------------------
    #-----LOOP THROUGH BASINS------------------------------------------------------------------------
    #-------------------------------------------------------------------------------------------------

    for ib, basin_0str in enumerate(list(sacsma_results_0.keys())): 
        basin_int = int(basin_0str)
        print(basin_0str)
        
        #-------------------------------------------------------------------------------------------------
        # Get the NWM data for this basin in an xarray dataset.
        xr_nwm = xr.DataArray(nwm_results[basin_0str]['streamflow'].values, 
                 coords=[nwm_results[basin_0str]['streamflow'].index], 
                 dims=['datetime'])
        #-------------------------------------------------------------------------------------------------


        #-------------------------------------------------------------------------------------------------
        # Get the HP training years for this basin.
        date_range_list = []
        for start_test_period, end_test_period in zip(hp_test_periods[basin_0str]['start_dates'],
                                                        hp_test_periods[basin_0str]['end_dates']):
            startx_time = min(max(start_test_period, pd.to_datetime('1996-10')), pd.to_datetime('2014-09'))
            endx_time = max(min(end_test_period, pd.to_datetime('2014-09')),pd.to_datetime('1996-10')) 
            if endx_time > startx_time:
                date_range = pd.date_range(startx_time,endx_time)
                date_range_list.extend(date_range)
        for start_val_period, end_val_period in zip(hp_val_periods[basin_0str]['start_dates'],
                                                        hp_val_periods[basin_0str]['end_dates']):
            startx_time = min(max(start_val_period, pd.to_datetime('1996-10')), pd.to_datetime('2014-09'))
            endx_time = max(min(end_val_period, pd.to_datetime('2014-09')),pd.to_datetime('1996-10')) 
            date_range = pd.date_range(startx_time,endx_time)
            if endx_time > startx_time:
                date_range = pd.date_range(startx_time,endx_time)
                date_range_list.extend(date_range)
        if len(date_range_list) < 1:
            continue
        #-------------------------------------------------------------------------------------------------        

        #-------------------------------------------------------------------------------------------------
        # Setting up the dictionary for the single basin results. Then will add to the overall dict.
        analysis_dict = {metric:{model:[] for model in models} for metric in metrics.get_available_metrics()}
        #-------------------------------------------------------------------------------------------------



        #-------------------------------------------------------------------------------------------------
        # We need the basin area to convert to CFS, to interpolate the RI from LPIII
        basin_area = pd_attributes.loc[basin_int, 'area_geospa_fabric']
        basin_str = tools.gauge_id_str(basin_int)
        #-------------------------------------------------------------------------------------------------


        #-------------------------------------------------------------------------------------------------
        # Make dictionary with all the flows
        flow_mm = {}
        #-------------------------------------------------------------------------------------------------


        #-------------------------------------------------------------------------------------------------        
         # NWM data
        sim_nwm = xr_nwm.loc[date_range_list]
        # convert from CFS to mm/day
        # fm3/s * 3600 sec/hour * 24 hour/day / (m2 * mm/m)
        flow_mm['nwm'] = sim_nwm*3600*24/(basin_area*1000)
        #-------------------------------------------------------------------------------------------------
        # Standard LSTM data trained on all years
        xrr = lstm_results_0[basin_0str]['1D']['xr']['QObs(mm/d)_sim']
        flow_mm['lstm'] = pd.DataFrame(data=xrr.values,index=xrr.date.values).loc[date_range_list]
        #-------------------------------------------------------------------------------------------------        
        # Mass Conserving LSTM data
        xrr = mclstm_results_0[basin_0str]['1D']['xr']['QObs(mm/d)_sim']
        flow_mm['mc'] = pd.DataFrame(data=xrr.values,index=xrr.date.values).loc[date_range_list]
        #-------------------------------------------------------------------------------------------------
        # SACSMA Sinlge run
        df = sacsma_results_0[basin_0str]
        flow_mm['sac'] = df.loc[date_range_list]
        #-------------------------------------------------------------------------------------------------
        # OBSERVATIONS
        xrr = mclstm_results_0[basin_0str]['1D']['xr']['QObs(mm/d)_obs']
        flow_mm['obs'] = pd.DataFrame(data=xrr.values,index=xrr.date.values).loc[date_range_list]
        #-------------------------------------------------------------------------------------------------


        #-------------------------------------------------------------------------------------------------
        # Make all xarray data similar
        for iflow in flows:
            if iflow == 'nwm': #already in the correct format
                continue
            if iflow == 'sac': #already in the correct format
                flow_mm[iflow] = xr.DataArray(np.array(flow_mm[iflow].values, dtype='float32'), 
                               coords=dict(datetime=flow_mm[iflow].index.values), dims=['datetime'])
            else:
                flow_mm[iflow] = xr.DataArray(flow_mm[iflow].values[:,0], 
                               coords=dict(datetime=flow_mm[iflow].index.values), dims=['datetime'])
        #-------------------------------------------------------------------------------------------------


        #-------------------------------------------------------------------------------------------------
        calculate_all_metrics_for_frequency_analysis(analysis_dict, flow_mm)
        #-------------------------------------------------------------------------------------------------


        #-------------------------------------------------------------------------------------------------     
        if breaktheloop:
            break
        #-------------------------------------------------------------------------------------------------     

        #-------------------------------------------------------------------------------------------------
        #Now that the basin has been analyzed successfully, add it to the larger dictionary
        analysis_dict_all[basin_0str] = analysis_dict
        #------------------------------------------------------------------------------------------------- 


    #-------------------------------------------------------------------------------------------------
    if dump_data:
        with open(file_name, 'wb') as fb:
            pkl.dump(analysis_dict_all, fb)

01434025


/home/jmframe/programs/anaconda3/envs/mc-paper/lib/python3.7/site-packages/scipy/stats/stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/home/jmframe/programs/anaconda3/envs/mc-paper/lib/python3.7/site-packages/scipy/stats/stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())


01047000
01170100
01365000
01139000
01350140


/home/jmframe/programs/anaconda3/envs/mc-paper/lib/python3.7/site-packages/scipy/stats/stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/home/jmframe/programs/anaconda3/envs/mc-paper/lib/python3.7/site-packages/scipy/stats/stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())


01195100
01137500
01484100
01423000
01350000
01162500
01187300


...

/home/jmframe/programs/anaconda3/envs/mc-paper/lib/python3.7/site-packages/scipy/stats/stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/home/jmframe/programs/anaconda3/envs/mc-paper/lib/python3.7/site-packages/scipy/stats/stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())


14020000
13331500
14096850


/home/jmframe/programs/anaconda3/envs/mc-paper/lib/python3.7/site-packages/scipy/stats/stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/home/jmframe/programs/anaconda3/envs/mc-paper/lib/python3.7/site-packages/scipy/stats/stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())


14137000
14138870
14138800
14138900
14139800
14141500
14154500
14166500
14158790
14185000
14182500
14185900
14187000
14216500
14222500
14236200
14301000
14303200
14305500
14306500
14306340
14308990
14309500
14325000
14316700
14362250
14400000


/home/jmframe/programs/anaconda3/envs/mc-paper/lib/python3.7/site-packages/scipy/stats/stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/home/jmframe/programs/anaconda3/envs/mc-paper/lib/python3.7/site-packages/scipy/stats/stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())


In [12]:
analysis_dict = {metric:{model:[] for model in models} for metric in metrics.get_available_metrics()}
for ib, basin_0str in enumerate(basin_list):
    try:
        for metric in metrics.get_available_metrics():
            for model in models:
                analysis_dict[metric][model].extend(analysis_dict_all[basin_0str][metric][model])
    except:
        continue

In [13]:
mc_mean_nse = []
for i in analysis_dict['NSE']['mc']:
    if i > -float("inf"):
        mc_mean_nse.append(i)
np.nanmean(mc_mean_nse)

0.7566589548594641

In [15]:
table_metrics = ['NSE','KGE','Pearson-r','Alpha-NSE','Beta-NSE','FHV','FLV','FMS','Peak-Timing']
print('NSE (mean), {}, {}, {}, {}'.format(np.round(np.nanmean(analysis_dict['NSE']['lstm']),4),
                                          np.round(np.nanmean(analysis_dict['NSE']['mc']),4),
                                          np.round(np.nanmean(analysis_dict['NSE']['sac']),4),
                                          np.round(np.nanmean(analysis_dict['NSE']['nwm']),4)))
for metric in table_metrics:
    print('{} (median), {}, {}, {}, {}'.format(metric,
                                          np.round(np.nanmedian(analysis_dict[metric]['lstm']),4),
                                          np.round(np.nanmedian(analysis_dict[metric]['mc']),4),
                                          np.round(np.nanmedian(analysis_dict[metric]['sac']),4),
                                          np.round(np.nanmedian(analysis_dict[metric]['nwm']),4)))

NSE (mean), 0.8062, -inf, 0.5789, 0.5493
NSE (median), 0.8502, 0.7988, 0.6525, 0.6694
KGE (median), 0.8118, 0.7589, 0.5786, 0.6355
Pearson-r (median), 0.9338, 0.9135, 0.8464, 0.8527
Alpha-NSE (median), 0.8502, 0.8099, 0.7004, 0.7928
Beta-NSE (median), -0.0201, -0.0239, 0.0346, -0.0399
FHV (median), -14.8644, -18.8628, -28.2913, -19.6384
FLV (median), 9.9204, -31.0659, 51.7205, 9.721
FMS (median), -8.3723, -6.759, -31.0643, -4.9286
Peak-Timing (median), 0.25, 0.2857, 0.5, 0.4444


In [87]:
# 475 results before adding NWM back. Not sure why I didn't include NWM>.. 

NSE (mean), 0.8062, -inf, 0.5789
NSE (median), 0.8502, 0.7988, 0.6525
KGE (median), 0.8118, 0.7589, 0.5786
Pearson-r (median), 0.9338, 0.9135, 0.8464
Alpha-NSE (median), 0.8502, 0.8099, 0.7004
Beta-NSE (median), -0.0201, -0.0239, 0.0346
FHV (median), -14.8644, -18.8628, -28.2913
FLV (median), 9.9204, -31.0659, 51.7205
FMS (median), -8.3723, -6.759, -31.0643
Peak-Timing (median), 0.25, 0.2857, 0.5


In [52]:
# These are the metrics across all the training data.

NSE (mean), 0.8991, 0.8169, 0.6168
NSE (median), 0.9103, 0.8363, 0.681
KGE (median), 0.9103, 0.8551, 0.6739
Pearson-r (median), 0.955, 0.9161, 0.8452
Alpha-NSE (median), 0.9323, 0.8968, 0.8198
Beta-NSE (median), -0.0053, -0.0023, 0.0723
FHV (median), -7.0299, -10.7159, -15.8229
FLV (median), -2.6546, -28.9755, 44.3396
FMS (median), -8.214, -7.8432, -35.7693
Peak-Timing (median), 0.2424, 0.3043, 0.4231


In [20]:
# These were the metrics across all years. 

NSE (mean), 0.8518, 0.7856, 0.5936
NSE (median), 0.8775, 0.8147, 0.6584
KGE (median), 0.8736, 0.8304, 0.6275
Pearson-r (median), 0.9383, 0.9047, 0.8348
Alpha-NSE (median), 0.9031, 0.8745, 0.8043
Beta-NSE (median), -0.0052, 0.001, 0.0806
FHV (median), -9.472, -12.1794, -16.3454
FLV (median), -7.778, -27.8374, 43.6461
FMS (median), -9.1512, -8.6821, -35.2811
Peak-Timing (median), 0.2571, 0.3077, 0.4333


In [21]:
np.round(np.nanmean(analysis_dict['NSE']['mc']),4)

0.7856